# 1. Install Dependencies

pip install torch from https://pytorch.org/ based on your specs

In [ ]:
#install YOLOv5
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt

# 2. Realtime Gauze Detection w/ 2 Webcams

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time

# YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/best3.pt', force_reload=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5x6', force_reload=True)

model.conf = 0.50
model.max_det = 20
model.classes = [0]

model2.conf = 0.25
model2.max_det = 20
model2.classes = [0]

onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()

    # Make detections
    pic = np.hstack((frame1, frame2))
    results = model(np.hstack((frame1, frame2)))
    results2 = model2(np.hstack((frame1, frame2)))

    a, b, c, d = 0, 0, 0, 0
    for detection in results.xyxy[0]:

        # Gauze detection
        if detection[5].item() in model.classes:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                a += 1
            elif detection[2].item() > frame1.shape[1]:
                b += 1

    for detection in results2.xyxy[0]:
        # Hand detection
        if detection[5].item() == 0:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                c += 1
            elif detection[2].item() > frame1.shape[1]:
                d += 1

    image = np.squeeze(results.render())

    # Adding black border at the bottom
    borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - frame1.shape[1], 640 - frame2.shape[1], cv2.BORDER_CONSTANT,
                                       value=[0, 0, 0])

    if c == 0:
        tempcount1 = a
        if tempcount1 != onscreenIn:
            frameCountIn += 1
            if frameCountIn == condition:
                if tempcount1 > onscreenIn:
                    countIn += tempcount1 - onscreenIn
                onscreenIn = tempcount1
        else:
            frameCountIn = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

    if d == 0:
        tempcount2 = b
        if tempcount2 != onscreenOut:
            frameCountOut += 1
            if frameCountOut == condition:
                if tempcount2 > onscreenOut:
                    countOut += tempcount2 - onscreenOut
                onscreenOut = tempcount2
        else:
            frameCountOut = 0
        cv2.putText(image, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(image, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2,
                    cv2.LINE_AA)
    else:
        cv2.putText(image, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

    # FPS Counter
    endTime = time.time()
    fps = 1 / (endTime - startTime)
    startTime = time.time()
    cv2.putText(borderedFrame, "FPS =" + str(round(fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                2, cv2.LINE_AA)

    # Gauze In-Play Counter
    countPlay = countIn - onscreenIn - countOut
    cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
                cv2.LINE_AA)

    cv2.imshow('Gauze', borderedFrame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()